In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [22]:
file = open("cfg/yolov3.cfg", "r")
lines = file.read().split('\n')
lines = [x for x in lines if len(x) > 0]
lines = [x for x in lines if x[0] != "#"]
lines = [x.rstrip().lstrip() for x in lines]

In [26]:
def parse_cfg(cfgfile="cfg/yolov3.cfg"):
    file = open(cfgfile, "r")
    lines = file.read().split('\n')
    lines = [x for x in lines if len(x) > 0]
    lines = [x for x in lines if x[0] != "#"]
    lines = [x.rstrip().lstrip() for x in lines]
    
    block = {}
    blocks = []

    for line in lines:
        if line[0] == "[":
            if len(block) != 0:
                blocks.append(block)
                block = {}
            block["type"] = line[1:-1].rstrip()
        else:
            key, value = line.split("=")
            block[key.rstrip()] = value.lstrip()
        blocks.append(block)
    return blocks

In [27]:
blocks = parse_cfg()

In [64]:
blocks[:]

[{'angle': '0',
  'batch': '1',
  'burn_in': '1000',
  'channels': '3',
  'decay': '0.0005',
  'exposure': '1.5',
  'height': '416',
  'hue': '.1',
  'learning_rate': '0.001',
  'max_batches': '500200',
  'momentum': '0.9',
  'policy': 'steps',
  'saturation': '1.5',
  'scales': '.1,.1',
  'steps': '400000,450000',
  'subdivisions': '1',
  'type': 'net',
  'width': '416'},
 {'angle': '0',
  'batch': '1',
  'burn_in': '1000',
  'channels': '3',
  'decay': '0.0005',
  'exposure': '1.5',
  'height': '416',
  'hue': '.1',
  'learning_rate': '0.001',
  'max_batches': '500200',
  'momentum': '0.9',
  'policy': 'steps',
  'saturation': '1.5',
  'scales': '.1,.1',
  'steps': '400000,450000',
  'subdivisions': '1',
  'type': 'net',
  'width': '416'},
 {'angle': '0',
  'batch': '1',
  'burn_in': '1000',
  'channels': '3',
  'decay': '0.0005',
  'exposure': '1.5',
  'height': '416',
  'hue': '.1',
  'learning_rate': '0.001',
  'max_batches': '500200',
  'momentum': '0.9',
  'policy': 'steps',
  '

In [48]:
class EmptyLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
class DetectionLayer(nn.Module):
    def __init__(self, anchors):
        super().__init__()
        self.anchors = anchors

In [53]:
def create_modules(blocks):
    net_info = blocks[0]
    module_list = nn.ModuleList()
    prev_filters = 3
    filters = prev_filters
    output_filters = []
    
    for index, x in enumerate(blocks[1:]):
        module = nn.Sequential()
        if x["type"] == "convolutional":
            activation = x["activation"]
            try:
                batch_normalize = int(x["batch_normalize"])
                bias = False
            except:
                batch_normalize = 0
                bias = True
            
            filters= int(x["filters"])
            padding = int(x["pad"])
            kernel_size = int(x["size"])
            stride = int(x["stride"])

            if padding:
                pad = (kernel_size - 1) // 2
            else:
                pad = 0
            
            # Add the convolutional layer
            conv = nn.Conv2d(prev_filters, filters, kernel_size,
                             stride, pad, bias=bias)
            module.add_module(f"conv_{index}", conv)
            
            # Add the batch normalization layer
            if batch_normalize:
                bn = nn.BatchNorm2d(filters)
                module.add_module(f"batch_norm_{index}", bn)
            
            # Check the activation
            if activation == "leaky":
                activation_fn = nn.LeakyReLU(0.1, inplace=True)
                module.add_module(f"leaky_{index}", activation_fn)
                
        # Upsamapling Layer
        elif x["type"] == "upsample":
            stride = int(x["stride"])
            upsample = nn.Upsample(scale_factor=2, mode="bilinear")
            module.add_module(f"upsample_{index}", upsample)
            
        # Route Layer
        elif x["type"] == "route":
            if not isinstance(x["layers"], list):
            x["layers"] = x["layers"].split(",")
            start = int(x["layers"][0])
            try:
                end = int(x["layers"][1])
            except:
                end = 0
            # Positive annotation
            if start > 0:
                start = start - index
            if end > 0:
                end = end - index
            route = EmptyLayer()
            module.add_module(f"route_{index}", route)
            if end < 0:
                filters = output_filters[index + start]\
                    + output_filters[index + end]
            else:
                filters = output_filters[index + start]
        
        # Shortcut corresponds to skip connection
        elif x["type"] == "shortcut":
            shortcut = EmptyLayer()
            module.add_module(f"shortcut_{index}", shortcut)
            
        # YOLO Layer
        elif x["type"] == "yolo":
            mask = x["mask"].split(",")
            mask = [int(x) for x in mask]
            
            anchors = x["anchors"].split(",")
            anchors = [int(a) for a in anchors]
            anchors = [(anchors[i], anchors[i+1]) for i in range(0, len(anchors),2)]
            anchors = [anchors[i] for i in mask]
            
            detection = DetectionLayer(anchors)
            module.add_module(f"Detection_{index}", detection)
            
        module_list.append(module)
        prev_filters = filters
        output_filters.append(filters)
        
    return net_info, module_list

In [54]:
blocks = parse_cfg()
print(create_modules(blocks))

AttributeError: 'list' object has no attribute 'split'

In [62]:
blocks[500]["layers"]

KeyError: 'layers'